In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%%capture
!pip install ftfy translate langdetect imbalanced-learn collections-extended rake_nltk googletrans

In [4]:
import sys
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks')

In [6]:
# !pip install chart_studio
# !pip install wordcloud
# !pip install ftfy
# !pip install rake_nltk
# !pip install gensim 
# !pip install missingno
# !pip install googletrans
# !pip install langdetect
# !pip install imblearn
# !pip install collections2

# Standard Library
import pandas as pd
import numpy as np

from ftfy import *
import nltk
from rake_nltk import Rake

import missingno as msno
import matplotlib.pyplot as plt 
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns 

# For Translation
from googletrans import Translator
from langdetect import detect

#For Upsampling
import imblearn
from collections import Counter
from matplotlib import pyplot
from numpy import where
# from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

# Import label encoder 
from sklearn import preprocessing 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer

# Import Dependencies

import os
import re

import tensorflow as tf
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

import keras

from keras import layers
# For Embedding Layer
from keras.layers.embeddings import Embedding

# For RNN Layer
from keras.models import Sequential
from keras.layers import Embedding, GRU, LSTM, Bidirectional
from keras.layers import Dense, Dropout, Activation, Input,  Flatten

# import numpy as np
# from tensorflow import keras
# from tensorflow.keras import layers

# For CNN Layer
from keras.layers import Conv1D, MaxPooling1D

# For Text Summarization
from gensim.summarization import summarize

%tensorflow_version 1.x

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [7]:

filename= "/content/drive/My Drive/Colab Notebooks/final_dataframe_forDeep_Learning_Model.csv"
df = pd.read_csv(filename)

# COPY THE BELOW CODE

In [8]:
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
df['Assignment group'] = label_encoder.fit_transform(df['Assignment group']) 
  
df['Assignment group'].unique() 

array([ 8, 30, 41, 52, 63, 74, 79, 80,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 34, 35, 36,
       37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 64, 65, 33, 66, 67, 68, 69, 70, 71, 72,
       73, 75, 76, 77, 78,  2,  3,  0,  1,  4,  6,  5,  7])

In [9]:
MAX_LENGTH = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.Complete_Description.values)
post_seq = tokenizer.texts_to_sequences(df.Complete_Description.values)
post_seq_padded = pad_sequences(post_seq, maxlen=MAX_LENGTH)

In [10]:
y = df['Assignment group']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(post_seq_padded, y, test_size=0.3, random_state=1)

In [12]:
X_train.shape,X_test.shape

((36560, 300), (15669, 300))

In [13]:
y_train.shape, y_test.shape

((36560,), (15669,))

In [14]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

16390

In [15]:
num_class = len(np.unique(df['Assignment group'].values))
num_class

81

## GRU

In [16]:
max_features = vocab_size  
maxlen = 200
seq_output_size = 128
filters = 64
kernel_size = 5
pool_size = 4
activation_func = 'sigmoid'

In [17]:
# GRU 
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
# x = layers.Add(Sequential())(inputs)
x = layers.Embedding(max_features, 128)(inputs)
x = layers.Dropout(0.2)(x)
x = layers.Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1)(x)
x = layers.MaxPooling1D(pool_size=pool_size)(x)
# Add GRU
x = layers.GRU(seq_output_size, dropout=0.2, recurrent_dropout=0.2)(x)
x = layers.Dense(1)(x)
# outputs = layers.Activation('sigmoid')(x)
outputs = layers.Dense(num_class, activation='softmax')(x)
# Add a classifier
gru_model = keras.Model(inputs, outputs)
gru_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 128)         2097920   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 64)          41024     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               74112     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129 

In [18]:
gru_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
gru_model.fit(X_train, y_train, batch_size=20, epochs=7, validation_data=(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 36560 samples, validate on 15669 samples
Epoch 1/7
36560/36560 [==============================] - 218s 6ms/step - loss: 3.8932 - acc: 0.0304 - val_loss: 3.5902 - val_acc: 0.0370
Epoch 2/7
36560/36560 [==============================] - 224s 6ms/step - loss: 3.3918 - acc: 0.0574 - val_loss: 3.3524 - val_acc: 0.0766
Epoch 3/7
36560/36560 [==============================] - 224s 6ms/step - loss: 5.3663 - acc: 0.0694 - val_loss: 3.2893 - val_acc: 0.0776
Epoch 4/7
36560/36560 [==============================] - 227s 6ms/step - loss: 3.2825 - acc: 0.0843 - val_loss: 3.2138 - val_acc: 0.0876
Epoch 5/7
36560/36560 [==============================] - 227s 6ms/step - loss: 3.0990 - acc: 0.1079 - val_loss: 3.1456 - val_acc: 0.0934
Epoch 6/7
36560/36560 [==============================] - 222s 6ms/step - loss: 3.0088 - acc: 0.1236 - val_loss: 3.1147 - val_acc: 0.0958
Epoch 7/7
36560/36560 [==============================] - 220s 6ms/step - loss: 3.0330 - acc: 0.1290 - val_loss: 3.0645 - val_acc

In [19]:
gru_predicted = gru_model.predict(X_test)
gru_predicted = np.argmax(gru_predicted, axis=1)
accuracy_score(y_test, gru_predicted)

0.10945178377688429